In [2]:
import json
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import sys

from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
import torch

cuda = torch.device('cuda')
device = 'cuda'

## 0. Producing Duplicated Songs and preprocessing

In [6]:
path = "C:\\Users\\kswoo\\OneDrive\\바탕 화면\\EASE\\"

In [7]:
with open(os.path.join(path, 'train.json'), 'rt',encoding='UTF8') as json_file:
    data = json.load(json_file)

In [8]:
train_data = pd.DataFrame([])
train_tags = pd.DataFrame([])
for i in tqdm(range(100000, 115071)) : 
    the_length = len(data[i]['songs'])
    the_data = pd.DataFrame({'user_id' : np.repeat(data[i]['id'], the_length),
                         'music_id' : data[i]['songs'], 
                         'playlist_title' : np.repeat(data[i]['plylst_title'], the_length), 
                         'likes' : np.repeat(data[i]['like_cnt'], the_length), 
                         'update_dates' : np.repeat(data[i]['updt_date'], the_length)})
    the_data_2 = pd.DataFrame({'user_id' : [data[i]['id']], 
                               'tags' : [data[i]['tags']]})
    train_data = pd.concat([train_data, the_data], axis = 0)
    train_tags = pd.concat([train_tags, the_data_2])

100%|████████████████████████████████████████████████████████████████████████████| 15071/15071 [10:10<00:00, 24.70it/s]


In [9]:
song_info = pd.read_csv('song_information.csv').iloc[:, 1:]
print(train_data.head(3))
song_info.head(3)

   user_id  music_id playlist_title  likes             update_dates
0   124325    112418          스탠리비치      7  2015-10-19 23:37:34.000
1   124325    363240          스탠리비치      7  2015-10-19 23:37:34.000
2   124325     75991          스탠리비치      7  2015-10-19 23:37:34.000


,song_id,song_name,artist_id,artist_name,song_genre,album_name,album_id,issue_date
0,0,Feelings,[2727],Various Artists,['GN0900'],불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,20140512
1,1,"Bach : Partita No. 4 In D Major, BWV 828 - II....",[29966],Murray Perahia,['GN1600'],"Bach : Partitas Nos. 2, 3 & 4",376431,20080421
2,2,Solsbury Hill (Remastered 2002),[3361],Peter Gabriel,['GN0900'],Hit,4698747,20180518


In [20]:
song_name = song_info.song_name.values
artist_name = song_info.artist_name.values
needed_key = song_info.groupby(by = ['song_name'], as_index = False).count()[song_info.groupby(by = ['song_name'], as_index = False).count().song_id  > 1].song_name.values

In [25]:
interest_keys = song_info[song_info['song_name'].isin(needed_key)].song_id.values
interest_keys

array([     0,      2,      4, ..., 707985, 707986, 707987], dtype=int64)

In [33]:
same_songs = []
finished = []
to_be_deleted = []

for i in tqdm(interest_keys) : 
    
    if i in finished : 
        to_be_deleted.append(i)
    
    else : 
        song_ = song_name[i]
        artist_ = artist_name[i]
        comp_lists = np.where(song_name == song_)[0]
        same_with = [i]
        
        if comp_lists.shape[0] == 1 : 
            None 
            
        else : 
            keys_ = np.setdiff1d(np.where(song_name == song_)[0], [i])
            for new_index in keys_ : 

                if artist_ == artist_name[new_index] : 

                    same_with.append(new_index)
                    finished.append(new_index)

                else : None
            
    
        same_songs.append(same_with)
        finished.append(i)

100%|████████████████████████████████████████████████████████████████████████| 324133/324133 [1:38:21<00:00, 54.93it/s]


In [35]:
#song_info[song_info['song_name'] == 'Walk Alone']
song_info.iloc[[0, 261506], :]

,song_id,song_name,artist_id,artist_name,song_genre,album_name,album_id,issue_date
0,0,Feelings,[2727],Various Artists,['GN0900'],불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,20140512
261506,261506,Feelings,[2727],Various Artists,['GN0900'],응답하라 7080 (추억의 빈티지 팝송여행),2228365,20140122


In [36]:
musics = train_data.music_id.values
edit_musics = musics.copy()
edit_musics

array([112418, 363240,  75991, ..., 640239,  13759, 154078], dtype=int64)

In [37]:
for dup in tqdm(same_songs) : 
    if len(dup) < 2 : None
    else : 
        for part in dup[1:] : 
            edit_musics[np.where(edit_musics == part)[0]] = dup[0]

100%|████████████████████████████████████████████████████████████████████████| 238552/238552 [00:34<00:00, 6953.44it/s]


In [38]:
train_data['new_music_id'] = edit_musics
new_train = train_data.drop(['update_dates'], axis = 1)
new_train.to_csv('new_train.csv')

## 1. 거르기

In [68]:
new_train = pd.read_csv('new_train.csv').iloc[:, 1:]
print(new_train.shape)
new_train.head(3)

(687187, 5)


,user_id,music_id,playlist_title,likes,new_music_id
0,124325,112418,스탠리비치,7,112418
1,124325,363240,스탠리비치,7,363240
2,124325,75991,스탠리비치,7,75991


In [69]:
def item_extractor(x, user_t, item_t) : 
    
    index_checking = x.groupby(by = ['new_music_id'], as_index = False).count()
    new_index = index_checking[index_checking.user_id >= item_t].new_music_id.values
    first = x[x['new_music_id'].isin(new_index)]
    
    index_checking2 = first.groupby(by = ['user_id'], as_index = False).count()
    new_2 = index_checking2[index_checking2.values >= user_t].user_id.values

    return first[first['user_id'].isin(new_2)]

In [70]:
new_x = item_extractor(new_train, 3, 3)
new_x.to_csv('final_x.csv', index = False)

## 2. 진짜 학습 시작

In [3]:
data = pd.read_csv('final_x.csv')
print(data.shape)
data.head(3)

(521087, 5)


,user_id,music_id,playlist_title,likes,new_music_id
0,124325,112418,스탠리비치,7,112418
1,124325,363240,스탠리비치,7,363240
2,124325,75991,스탠리비치,7,75991


In [4]:
user_enc = LabelEncoder()
item_enc = LabelEncoder()

user_index = user_enc.fit_transform(data.user_id.values).astype(int)
item_index = item_enc.fit_transform(data.new_music_id.values).astype(int)

In [5]:
print(user_index)
print(item_index)
print(np.unique(user_index).shape)
print(np.unique(item_index).shape)

[11999 11999 11999 ...  9692  9692  9692]
[10840 29550  7593 ...  1269  5301 15735]
(14725,)
(48652,)


In [6]:
indexs = torch.LongTensor([user_index, item_index])
indexs

tensor([[11999, 11999, 11999,  ...,  9692,  9692,  9692],
        [10840, 29550,  7593,  ...,  1269,  5301, 15735]])

In [7]:
#For the Normal CSR Matrix
X = csr_matrix((np.ones(user_index.shape[0]).astype(int), (user_index, item_index)))

In [7]:
# For torch
#X = torch.sparse.FloatTensor(indexs, 
#                             torch.ones(user_index.shape[0]), 
#                            torch.Size([np.unique(user_index).shape[0], np.unique(item_index).shape[0]]))

In [8]:
G = X.T.dot(X).toarray().astype(float)
G

array([[ 5.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., 20.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  4., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  3.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  3.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  3.]])

In [9]:
diagIndices = np.diag_indices(G.shape[0])
G[diagIndices] += 0.5 # Lambda

In [10]:
P = np.linalg.inv(G)
print('First Step Done!')
B = P / (-np.diag(P))
print('Second Step Done!')
B[diagIndices] = 0

First Step Done!
Second Step Done!


In [11]:
np.save('B_mat', B)

In [12]:
np.save('user_classes.npy', user_enc.classes_)
np.save('item_classes.npy', item_enc.classes_)

## 3. Getting the Trained Value

In [3]:
song_info = pd.read_csv('song_information.csv').iloc[:, 1:]
B = np.load('B_mat.npy')
user_enc = np.load('user_classes.npy')
item_enc = np.load('item_classes.npy')

In [5]:
np.where(item_enc == 37748)

(array([3958], dtype=int64),)

In [6]:
from musivisor import *

In [7]:
model = Musivisor('김선우', B, song_info, item_enc)

안녕하세요 김선우님 당신만을 위한 음악추천. Musivisor입니다.
지금부터 Musivisor 사용 방법을 설명드리겠습니다.
먼저, Musivisor는 4GB의 램을 필요로 합니다. 램이 부족하신경우, 다른 프로그램을 종료하여 충분한 램 공간을 확보해주세요.
1. 원하시는 곡 개수를 넣고 input_generator 함수를 시행해주세요. 이어 곡의 이름을 넣고 실행하시면, 가이드라인이 나옵니다.
2. input_generator 의 시행이 종료되면, prediction_generator를 시행해주세요!


In [8]:
model.input_generator(5)

너였다면


,Unnamed: 0,song_id,song_name,artist_id,artist_name,song_genre,album_name,album_id,issue_date
37748,37748,37748,너였다면,[792091],정승환,"['GN1500', 'GN0100']",또 오해영 OST Part.5,2687963,20160531


원하시는 곡이 있습니까? 있다면 Y, 없다면 N
Y

곡의 번호를 입력해주세요
37748
너였다면이 당신의 리스트에 추가되었습니다
다음 곡을 골라주세요.


피땀


,Unnamed: 0,song_id,song_name,artist_id,artist_name,song_genre,album_name,album_id,issue_date


원하시는 곡이 있습니까? 있다면 Y, 없다면 N
N
다른 노래를 입력해주세요.
피 땀


...

,Unnamed: 0,song_id,song_name,artist_id,artist_name,song_genre,album_name,album_id,issue_date
72499,72499,72499,처음처럼,[161636],엠씨더맥스 (M.C the MAX),['GN0100'],CEREMONIA,10408131,20200325
88249,88249,88249,처음처럼 사랑하길 바라고 있어,[424945],에스프레소,['GN0100'],좋은 느낌이 있어 너와 함께 있을 땐,2198949,20130830
148917,148917,148917,처음처럼,[3305],성시경,['GN0100'],처음처럼,2138,20010400
184658,184658,184658,처음처럼 그때처럼 (Feat. 강민경 From Davichi),[58793],이승기,['GN0100'],처음처럼 그때처럼,750153,20091222
219000,219000,219000,여전히 헤어짐은 처음처럼 아파서,[25],신승훈,['GN0100'],MY PERSONAS,10403337,20200408
245491,245491,245491,처음처럼,[412000],티아라,"['GN2500', 'GN0200']",Breaking Heart (Repackage),809262,20100223
468261,468261,468261,처음처럼 (Feat. July),[322766],메이다니 (MAYDONI),['GN0100'],7 Teen (Single),470352,20090115


원하시는 곡이 있습니까? 있다면 Y, 없다면 N
N
다른 노래를 입력해주세요.
폰서트


,Unnamed: 0,song_id,song_name,artist_id,artist_name,song_genre,album_name,album_id,issue_date
102980,102980,102980,폰서트,[468244],10CM,"['GN0500', 'GN0800']",4.0,10093077,20170901


원하시는 곡이 있습니까? 있다면 Y, 없다면 N
Y

곡의 번호를 입력해주세요
102980
폰서트이 당신의 리스트에 추가되었습니다
다음 곡을 골라주세요.


다음 음악들이 추천의 재료로 들어갑니다! ['너였다면', '피 땀 눈물', '나의 사춘기에게', '너를 만나', '폰서트']


[37748, 23360, 116114, 322491, 102980]

In [9]:
model.prediction_generator(10)

노래가 생성중입니다.. 잠시만 기다려주세요..

노래가 선택되었습니다! 추출중입니다..
김선우님. 오늘은 이런 음악 어떠세요?


,Unnamed: 0,song_id,song_name,artist_id,artist_name,song_genre,album_name,album_id,issue_date
144663,144663,144663,밤편지,[261143],아이유,['GN0100'],밤편지,10047890,20170324
165454,165454,165454,"모든 날, 모든 순간 (Every day, Every Moment)",[752425],폴킴,['GN0100'],`키스 먼저 할까요?` OST,10163394,20180502
288869,288869,288869,DNA,[672375],방탄소년단,"['GN2500', 'GN0300']",LOVE YOURSELF 結 `Answer`,10197480,20180824
305045,305045,305045,가을 아침,[261143],아이유,['GN0800'],꽃갈피 둘,10096855,20170922
362966,362966,362966,봄날,[672375],방탄소년단,"['GN2500', 'GN0300']",YOU NEVER WALK ALONE,10037969,20170213
449244,449244,449244,스토커,[468244],10CM,"['GN0500', 'GN0800']",3.0,2290452,20141119
478754,478754,478754,선물,[839732],멜로망스,"['GN0500', 'GN0100']",Moonlight,10078176,20170710
485640,485640,485640,열애중,[655051],벤,['GN0100'],RECIPE,10164839,20180508
516653,516653,516653,노력,[41499],박원,['GN0100'],1/24,10016012,20161117
531820,531820,531820,한숨,[646171],이하이,['GN0400'],SEOULITE,2671866,20160309
